### zadanie 2 
W tym zadaniu poeksperymentujemy z różnymi architekturami sieci neurnonowej - oraz różnymi optimizerami - zobaczymy jaki to ma wpływ na postęp uczenia

In [ ]:
# Wczytaj tę komórkę - zawiera funkcje potrzebne do wczytania / przetwarzania danych 

import numpy as np
from urllib import request
import gzip
import pickle
import torch 
from torch import nn
import pandas as pd 
from matplotlib import pyplot as plt 

import random 
def shuffle_data(x, y):
    c = list(zip(x, y))
    random.shuffle(c)
    return zip(*c)

filename = [
    ["training_images","train-images-idx3-ubyte.gz"],
    ["test_images","t10k-images-idx3-ubyte.gz"],
    ["training_labels","train-labels-idx1-ubyte.gz"],
    ["test_labels","t10k-labels-idx1-ubyte.gz"]
]

def download_mnist():
    base_url = "http://yann.lecun.com/exdb/mnist/"
    for name in filename:
        print("Downloading "+name[1]+"...")
        request.urlretrieve(base_url+name[1], name[1])
    print("Download complete.")

def save_mnist():
    mnist = {}
    for name in filename[:2]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1,28*28)
    for name in filename[-2:]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=8)
    with open("mnist.pkl", 'wb') as f:
        pickle.dump(mnist,f)
    print("Save complete.")

def init():
    download_mnist()
    save_mnist()

def preprocess(x):
    return [y.flatten()  for y in x]


def normalize(x):
    mean_val = np.mean(x)
    stdev_val = np.std(x)
    return (x - mean_val) / stdev_val

def binarize(y):
    return [int(elem == 5) for elem in y]
         

def load(n = 5000):
    with open("mnist.pkl",'rb') as f:
        mnist = pickle.load(f)
    return (
        normalize(preprocess(mnist["training_images"][:n]  / 255.)), 
        [x[0] for x in preprocess(mnist["training_labels"][:n])], 
        normalize(preprocess(mnist["test_images"][:n]  / 255.)), 
        [x[0] for x in preprocess(mnist["test_labels"][:n])]
    )

init()

In [ ]:
# wczytaj tę komórkę. 
x_train, y_train, x_test, y_test = load()
x_train, y_train = zip(*[x for x in list(zip(x_train, y_train)) if x[1] == 5 or (random.random() < 0.12)])
x_test, y_test = zip(*[x for x in list(zip(x_test, y_test)) if x[1] == 5 or (random.random() < 0.12)])

y_train_bin = binarize(y_train)
y_test_bin = binarize(y_test) 


In [ ]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self):
        super().__init__()        
        self.mlp = nn.Sequential(
            nn.Linear(784, 50),
            nn.Sigmoid(),
            nn.Linear(50,15),
            nn.Sigmoid(),
            nn.Linear(15, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):        
        return self.mlp(x)


In [ ]:
# training loop - pętla trenująca została zamknięta w funkcję train 

from tqdm import tqdm
def training_loop(model, optimizer, x_train, y_train, batch_size = 128, epochs=50):
    losses = []
    bce_loss = nn.BCELoss()
    for epoch in tqdm(range(epochs)):
        x, y = shuffle_data(x_train,y_train_bin)
        current_index = 0 
        while current_index < len(x_train):
            batch_x = x[current_index:(current_index + batch_size)]
            batch_y = y[current_index: (current_index + batch_size)] # 
            
            tensor_batch_x = torch.Tensor(batch_x)
            tensor_batch_y = torch.Tensor(batch_y).reshape(-1, 1)
            
            pred = model.forward(tensor_batch_x)
            loss = bce_loss(pred, tensor_batch_y)

                
            # Backpropagation
            loss.backward() # Liczenie gradientu wag modelu
            optimizer.step() # adam oblicza nowe parametry sieci 
            optimizer.zero_grad()
            
            current_index += batch_size
            losses.append(float(loss.detach().numpy()))
            
    return model, losses  # funkcja zwraca dwie wartości: nauczony model oraz listę losses(błędów) w każdej iteracji  


In [ ]:
# Task 1.
model = MultiLayerPerceptron()
optimizer = None # TODO: wypróbuj optimizer Adam : torch.optim.Adam
trained_model, losses_adam = training_loop(model=model, optimizer = optimizer, x_train=x_train, y_train=y_train_bin, batch_size=256)


In [ ]:
# Task 2.
model = MultiLayerPerceptron()
optimizer = None  # TODO: wypróbuj optimzier SGD torch.optim.SGD poeksperymentuj z parametrem lr= [wypróbuj wartości 0.01, 0.05, 0.1, 0.2]
trained_model, losses_sgd = training_loop(model=model, optimizer = optimizer, x_train=x_train, y_train=y_train_bin, batch_size=256)

In [ ]:
# Task 3.
model = MultiLayerPerceptron()
optimizer = None # TODO: wypróbuj AdamW torch.optim.AdamW 
trained_model,losses_adamw = training_loop(model=model, optimizer = optimizer, x_train=x_train, y_train=y_train_bin, batch_size=256)


In [ ]:
# Task4. 
# TODO: Pokaż jak loss/error wszystkich trzech optimizerów zmienia się w czasie



# pd.Series(losses_adam).plot(label='adam')
# pd.Series(losses_adamw).plot(label='adamw')
# pd.Series(losses_sgd).plot(label='sgd')
# plt.legend()

In [ ]:
# Task 5: Poeksperymentuj z różnymi parami sieci neuronowych i zobacz jak zmienia się loss po zmianie architektury sieci oraz funkcji aktywacji 

# Twój oryginalny perceptron 
class MultiLayerPerceptronA(nn.Module):
    def __init__(self):
        super().__init__()        
        self.mlp = nn.Sequential(
            nn.Linear(784, 50),
            nn.Sigmoid(),
            nn.Linear(50,15),
            nn.Sigmoid(),
            nn.Linear(15, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):        
        return self.mlp(x)

# Perceptron z którym go porównasz 
class MultiLayerPerceptronB(nn.Module):
    def __init__(self):
        super().__init__()        
        self.mlp = nn.Sequential(
            nn.Linear(784, 50), # TODO: Poeksperymentuj z różną ilością neuronów 
            nn.Sigmoid(), # TODO: Zamień aktywacje na ReLU 
            nn.Linear(50, 15), # TODO: Poeksperymentuj z różną ilością neuronów 
            nn.Sigmoid(),  # TODO: Zamień aktywacje na ReLU 
            nn.Linear(15, 1), # TODO: Poeksperymentuj z różną ilością neuronów 
            nn.Sigmoid() 
        )
    
    def forward(self, x):        
        return self.mlp(x)


In [ ]:
model = MultiLayerPerceptronA()
optimizer = torch.optim.Adam(model.parameters())
losses_a = training_loop(model=model, optimizer = optimizer, x_train=x_train, y_train=y_train_bin, batch_size=256)


In [ ]:
model = MultiLayerPerceptronB()
optimizer = torch.optim.Adam(model.parameters())
losses_b = training_loop(model=model, optimizer = optimizer, x_train=x_train, y_train=y_train_bin, batch_size=256)


In [ ]:
# Proces uczenia dla powyższych par A, B 
import pandas as pd 
from matplotlib import pyplot as plt 
pd.Series(losses_a).plot(label='A')
pd.Series(losses_b).plot(label='B')
plt.legend()

In [1]:
# Task 6. 
# TODO (dla chętnych): Zaimplementuj funkcje które wyliczają 2 wartości:
# https://en.wikipedia.org/wiki/Sensitivity_and_specificity
# True Positive rate 
# False Positive rate  

# Są to kolejne wazne metryki których używamy do mierzenia jakości rozwiązania problemu klasyfikacji binarnej 
# Spróbuj zaimplementować je i zmierzyć te wartości dla różnych thresholdów któregoś z wyuczonych modeli 